In [3]:
# !pip install sonnet
# !pip install graphs
# !pip uninstall sonnet
# !pip install dm-sonnet

In [1]:
import numpy as np
import tensorflow as tf
import pickle
import os
from plot_region.tools import load_mnist, random_string, get_sample_plane3, count_regions_2d

2024-06-06 14:43:34.740377: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 14:43:50.222609: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 14:43:50.322005: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 14:44:14.151260: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
TRAIN = ('./data/MNIST/raw/train-images-idx3-ubyte',
         './data/MNIST/raw/train-labels-idx1-ubyte')
TEST = ('./data/MNIST/raw/t10k-images-idx3-ubyte',
        './data/MNIST/raw/t10k-labels-idx1-ubyte')
NETWORKS = [[16, 16, 16, 16], [32, 32, 32, 32], [64, 64, 64, 64], [20, 20, 20], [40, 40, 40], [60, 60, 60], [80, 80, 80]]
BIAS_STD = 0.001
REPEATS = 5
LR = 1e-3
BATCH_SIZE = 128
EPOCHS = 20
NUM_SAMPLE = 5
REPORT_EPOCHS = [0, 0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 6, 8, 10, 15, 20]
HOME_DIR = './test_results'
OUTPUT_BASE = 'count_2d'

assert REPORT_EPOCHS[-1] == EPOCHS, 'REPORT_EPOCHS[-1] should equal EPOCHS'
output_dir = OUTPUT_BASE + "_epochs_%d_lr_%s_batch_size_%d_bias_std_%s_num_sample_%d_report_epochs_%s" %(
    EPOCHS, str(LR), BATCH_SIZE, str(BIAS_STD), NUM_SAMPLE, str(REPORT_EPOCHS))
if output_dir not in os.listdir(HOME_DIR):
    os.makedirs(HOME_DIR + output_dir)

for network in NETWORKS:
    if str(network) not in os.listdir(HOME_DIR + output_dir):
        os.makedirs(HOME_DIR + output_dir + '/' + str(network))


In [12]:
X_train, Y_train = load_mnist(TRAIN)
X_test, Y_test = load_mnist(TEST)
num_train = Y_train.shape[0]
num_test = Y_test.shape[0]

num_per_epoch = int(num_train / BATCH_SIZE)
report_batches = np.floor(num_per_epoch * np.array(REPORT_EPOCHS)).astype(int)

samples1 = np.random.choice(num_train, NUM_SAMPLE, replace=False)
samples2 = np.random.choice(num_train, NUM_SAMPLE, replace=False)
points1 = [X_train[sample1, :] for sample1 in samples1]
points2 = [X_train[sample2, :] for sample2 in samples2]
input_planes = []
for point1, point2 in zip(points1, points2):
    input_plane_weight, input_plane_bias, _, _ = get_sample_plane3(point1, point2)
    input_planes.append((input_plane_weight, input_plane_bias))

def count(sess, weights, biases, input_placeholder, output_placeholder):
    [the_weights, the_biases] = sess.run([weights, biases],
                                         feed_dict={input_placeholder: X_test, output_placeholder: Y_test})
    return [count_regions_2d(the_weights, the_biases, input_plane_weight, input_plane_bias,
                             input_vertices=np.array([[-20, -20], [20, -20], [20, 20], [-20, 20]]),) for
            (input_plane_weight, input_plane_bias) in input_planes]


In [ ]:
a, b = mnist_training_analysis([512, 16], pre_path='test_results/', epochs=5)

start training


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
scale = 1
normal_dist = True
archs = [
    # (100, [100]),
    # (100, [100, 100]),
    # (2, [4])
    (2, [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100])
]

In [ ]:
results = [one_random_experiment(i, exps=10, num=10000, one=False, pre_path='test_results/', normal_dist=normal_dist, loc=0, scale=scale, projection_analysis_bool=True) for i in tqdm(archs)]
print("DONE")

# [before_after_training_experiment(i, pre_path='test_results/', num=100, epochs=5, normal_dist=normal_dist, loc=0, scale=scale) for i in archs]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [09:56<00:00, 596.31s/it]

DONE


In [ ]:
# animate_histogram(results, 'hidden Layers', pre_path='test_results/')